In [13]:
import pandas as pd
import numpy as np
import dask as dd
from typing import Tuple
import shortuuid

In [14]:
malcom_data_sheet = pd.ExcelFile('../Malcom Feb 17 2020.xlsx')
existing_actors = pd.read_csv('actors.csv')
existing_organizations = pd.read_csv('organizations.csv')
relevant_sheets = ['Actors', 'Organizations', 'Positions' , 'Project']
to_lower_lists = ['Last Name',	'MI', 'First Name'	,'Name'	,'Short Name']
new_actors = {'full_name': [] , 'first_name':[] , 'last_name':[] , 'middle_initial':[] , 'act_id': []}
new_organization = {'name': [] , 'short_name':[] ,'mnemnoic':[] ,'category': [] , 'org_id': [], 'sector':[] , 'second_sector':[]}

In [15]:
dataframe_lists = {}
for i in relevant_sheets:
    dataframe_lists[i] = pd.read_excel(malcom_data_sheet , i)

dataframe_lists['Actors'] = dataframe_lists['Actors'].apply(lambda x : x.str.lower() if x.name in to_lower_lists else x)
dataframe_lists['Organizations'] = dataframe_lists['Organizations'].apply(lambda x: x.str.lower())
print(dataframe_lists['Actors'].dtypes)
print(dataframe_lists['Organizations'].dtypes)



Input Source             object
Last Name                object
MI                       object
First Name               object
Name                     object
ACTID                    object
Short Name               object
Rank                     object
Branch                   object
Entered service         float64
Retired from service    float64
Service Role             object
Retired or died         float64
dtype: object
Input Source    object
Name            object
Short name      object
ORGID           object
Category        object
Mnemnoic        object
Sector          object
Sector2         object
XML             object
Unnamed: 9      object
dtype: object


In [16]:
def create_actor(full_name:str, **kwargs)-> Tuple[bool , int]:
    first_name = str(kwargs.get('first_name' , None))
    last_name = str(kwargs.get('last_name' , None))
    middle_initial = str(kwargs.get('middle_initial' , None))
    if middle_initial is not None:
        middle_initial = middle_initial.strip('.,')

    id = shortuuid.ShortUUID().random(length=12)
    new_actors['full_name'].append(full_name)
    new_actors['first_name'].append(first_name)
    new_actors['last_name'].append(last_name)
    new_actors['middle_initial'].append(middle_initial)
    new_actors['act_id'].append(id)
    return id


def check_actor_exist(full_name:str, **kwargs)-> Tuple[bool , int]:
    actor = existing_actors.loc[existing_actors['full_name'] == full_name]
    first_name = kwargs.get('first_name' , None)
    last_name = kwargs.get('last_name' , None)
    middle_initial = kwargs.get('middle_initial' , None)
    if(actor.empty):
        if last_name != None:
            try:
                first_name.lower()
                actor = existing_actors.loc[existing_actors['last_name'] == last_name]
                if not actor.empty and first_name != None:
                    actor = actor.loc[actor['first_name'] == first_name]
                    if not actor.empty:
                        return (True , actor.get('act_id').to_numpy()[0])
                    else:
                        id = create_actor(full_name , first_name=first_name , last_name=last_name , middle_initial=middle_initial)
                        return (True, id)
                else:
                    id = create_actor(full_name , first_name=first_name , last_name=last_name , middle_initial=middle_initial)
                    return (True , id)
            except Exception as e:
                raise e
    elif(actor.shape[0] > 1):
        pass
    else:
        return (True , actor.get('act_id').to_numpy()[0])    


In [17]:
def check_all_actors(actors):
    try:
        print(actors.apply(lambda x : check_actor_exist(x['Name'] , first_name=x['First Name'] , last_name=x['Last Name'] , middle_initial=x['MI']) , axis=1))
        return (True , 1)
    except Exception as e: 
        return (False , -1 , e)

check_all_actors(dataframe_lists['Actors'])
new_actors = pd.DataFrame.from_dict(new_actors)
existing_actors = pd.concat([existing_actors , new_actors])



In [18]:
def create_organization(name:str , **kwargs)-> Tuple[bool, int]:
    short_name = kwargs.get('short_name', None)
    category = kwargs.get('category', None)
    sector = kwargs.get('sector', None)
    second_sector = kwargs.get('second_sector', None)
    mnemnoic = kwargs.get('mnemoic', None)
    org_id = shortuuid.ShortUUID().random(length=12)
    new_organization['name'].append(name)
    new_organization['short_name'].append(short_name)
    new_organization['category'].append(category)
    new_organization['sector'].append(sector)
    new_organization['second_sector'].append(second_sector)
    new_organization['mnemnoic'].append(mnemnoic)
    new_organization['org_id'].append(org_id)
    return id


def check_organization_exists(name:str , **kwargs)-> Tuple[bool , int]:
    organization = existing_organizations.loc[existing_organizations['name'] == name]
    short_name = kwargs.get('short_name', None)
    category = kwargs.get('category', None)
    sector = kwargs.get('sector', None)
    second_sector = kwargs.get('second_sector', None)
    mnemnoic = kwargs.get('mnemoic', None)
    if organization.empty:
            try:
                if short_name is not None:
                    organization = existing_organizations.loc[existing_organizations['short_name'] == short_name]
                    if not organization.empty:
                        return (True , organization.get('org_id').to_numpy()[0])
                if mnemnoic is not None: 
                    organization = existing_organizations.loc[existing_organizations['mnemonic'] == mnemonic]
                    if not organization.empty:
                        return (True , organization.get('org_id').to_numpty()[0])
                create_organization(name , short_name=short_name , category=category , sector=sector ,second_sector=second_sector , mnemonic=mnemnoic)
            except Exception as e:
                raise e
    else:
        return (True , organization.get('org_id').to_numpy()[0])

def check_all_orgs(organizations):
    try:
        organizations.apply(lambda x : check_organization_exists(x['Name'], short_name=x['Short name'] , category=x['Category'], sector=x['Sector'], second_sector=x['Sector2']) , axis=1)
        return (True , 1)
    except Exception as e:
        raise e
        


check_all_orgs(dataframe_lists['Organizations'])
new_organizations = pd.DataFrame.from_dict(new_organization)
existing_organizations = pd.concat([existing_organizations, new_organizations])